In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from imblearn.combine import SMOTEENN,SMOTETomek
from imblearn.ensemble import BalanceCascade
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from mlxtend.plotting import plot_decision_regions
import h5py
import keras
from sklearn.utils import class_weight
from keras.utils import to_categorical
from keras.optimizers import *
from keras.regularizers import *
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score, precision_recall_fscore_support

%matplotlib inline

KFOLD_SEED = 42

def shuffle(features, labels):
    p = np.random.permutation(len(features))
    return features[p], labels[p]

def rtb_confusion_matrix(test_labels, y_preds):
    m = confusion_matrix(test_labels[:,1], y_preds.argmax(axis=-1))
    
    print("================================")
    print("Confusion Matrix:")
    print("True Negative = %d" % m[0][0])
    print("False Negative = %d" % m[1][0])
    print("True Positive = %d" % m[1][1])
    print("False Positive = %d" % m[0][1])


def rtb_f1_score(test_labels, y_preds):
    f = f1_score(test_labels[:,1], y_preds.argmax(axis=-1))
    print("================================")
    print("f1 score = %0.3f" % f)

    
def print_metrics(true_labels, y_preds, is_train=True):
    if is_train:
        print("--------train---------")
    else:
        print("--------test---------")
    
    rtb_confusion_matrix(true_labels, y_preds)
    rtb_f1_score(true_labels, y_preds)
    print("================================")
    print("ROC AUC Score = %0.3f" % roc_auc_score(true_labels[:,1], y_preds.argmax(axis=-1)))

In [2]:
input_path = '~/data/biddings.csv'
data = pd.read_csv(input_path)
print(data.shape)

train = data[:800000]
test = data[800000:]

sample = train.sample(frac=1)
features = sample.drop('convert', axis=1).values
labels = to_categorical(sample.convert.ravel(), 2)

test_features = test.drop('convert', axis=1).values
test_labels = to_categorical(test.convert.ravel(), 2)

(1000000, 89)


In [3]:
# Data prep
'''
when sample weights is balanced, model predicts everything as positive. This is too strong
'''
sample_weights = class_weight.compute_sample_weight(
    class_weight={0:1, 1:100},
    y=labels[:,1])
class_weights = class_weight.compute_class_weight('balanced', np.unique(labels[:,1]), labels[:,1])

print(class_weights)

[  0.50095871 261.26714566]


In [4]:
batch_size = 32
epochs = 5

# Build model
def create_model():
    model = Sequential()
    model.add(Dense(64, input_shape=(88,),
                    activation='relu',
                    kernel_regularizer=l2(0.01),
                    kernel_initializer='glorot_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(32,
                    activation='relu',
                    kernel_regularizer=l2(0.01),
                    kernel_initializer='glorot_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(2, activation='softmax'))

    model.summary()

    model.compile(loss='binary_crossentropy',
    #               loss_weights=class_weights,
                  optimizer=SGD(lr=0.1, decay=0.02),
                  metrics=['accuracy'])
    return model

In [5]:
n_batches = 10

def gen_batches(features, labels, n_batches=10, ratio=10):
    positive_samples = sample[sample.convert == 1]
    negative_samples = sample[sample.convert == 0]
    
    positive_size = positive_samples.shape[0]
    
    print(positive_size)
    
#     batches = np.empty(shape=(n_batches, positive_size * 2, 89))
    feature_batches = np.empty(shape=(n_batches, positive_size*(ratio+1), 88))
    label_batches = np.empty(shape=(n_batches, positive_size*(ratio+1), 2))

    for i in range(n_batches):
        b = negative_samples[i*n_batches : (positive_size*ratio + i*n_batches)]
        shuffled = pd.concat([b, positive_samples]).sample(frac=1)
        shuffled_features = shuffled.drop('convert', axis = 1).values
        shuffled_labels = to_categorical(shuffled.convert.ravel(), 2)
        
        print(shuffled_features.shape, shuffled_labels.shape)
        np.append(feature_batches, shuffled_features)
        np.append(label_batches, shuffled_labels)
    
    return feature_batches, label_batches


def train_batched_models(feature_batches, label_batches):
    models = []
    for i, fb in enumerate(feature_batches):
        model = create_model()
        print(fb.shape)

        lb = label_batches[i]
        
        print(fb.shape, lb.shape)
    
        model.fit(fb, lb,
            batch_size=batch_size,
#             class_weight={0:1, 1:5},
    #       sample_weight=sample_weights,
            epochs=epochs,
            callbacks=[keras.callbacks.EarlyStopping()],
            validation_split=0.2,
            verbose=1)
        models.append(model)
    return models


def predict_batched_models(models, test_features, test_labels, n_batches=10):
    cum_preds = None
    for model in models:
        test_preds = model.predict(test_features, verbose=1)
        print(test_preds.shape)
        print(test_preds[0:10])
        print_metrics(test_labels, test_preds, is_train=False)
        
        if cum_preds is None:
            cum_preds = test_preds
            print(cum_preds.shape)
        else:
            cum_preds += test_preds
    
    return cum_preds/n_batches


feature_batches, label_batches = gen_batches(features, labels, n_batches=n_batches)
models = train_batched_models(feature_batches, label_batches)

1531
(16841, 88) (16841, 2)
(16841, 88) (16841, 2)
(16841, 88) (16841, 2)
(16841, 88) (16841, 2)
(16841, 88) (16841, 2)
(16841, 88) (16841, 2)
(16841, 88) (16841, 2)
(16841, 88) (16841, 2)
(16841, 88) (16841, 2)
(16841, 88) (16841, 2)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                5696      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 7,842
Trainable params: 7

Train on 13472 samples, validate on 3369 samples
Epoch 1/5
13472/13472 [==============================] - 1s 101us/step - loss: 1.5761 - acc: 1.0000 - val_loss: 1.4497 - val_acc: 1.0000
Epoch 2/5
13472/13472 [==============================] - 1s 73us/step - loss: 1.3987 - acc: 1.0000 - val_loss: 1.3589 - val_acc: 1.0000
Epoch 3/5
13472/13472 [==============================] - 1s 71us/step - loss: 1.3323 - acc: 1.0000 - val_loss: 1.3094 - val_acc: 1.0000
Epoch 4/5
13472/13472 [==============================] - 1s 75us/step - loss: 1.2918 - acc: 1.0000 - val_loss: 1.2760 - val_acc: 1.0000
Epoch 5/5
13472/13472 [==============================] - 1s 74us/step - loss: 1.2631 - acc: 1.0000 - val_loss: 1.2512 - val_acc: 1.0000
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 64)                5696      
_________________________________________________________________
dropou

Train on 13472 samples, validate on 3369 samples
Epoch 1/5
13472/13472 [==============================] - 2s 131us/step - loss: 1.5727 - acc: 0.0093 - val_loss: 1.4467 - val_acc: 0.0102
Epoch 2/5
13472/13472 [==============================] - 1s 70us/step - loss: 1.3959 - acc: 0.0095 - val_loss: 1.3563 - val_acc: 0.0120
Epoch 3/5
13472/13472 [==============================] - 1s 71us/step - loss: 1.3298 - acc: 0.0088 - val_loss: 1.3069 - val_acc: 0.0120
Epoch 4/5
13472/13472 [==============================] - 1s 71us/step - loss: 1.2894 - acc: 0.0096 - val_loss: 1.2737 - val_acc: 0.0120
Epoch 5/5
13472/13472 [==============================] - 1s 71us/step - loss: 1.2608 - acc: 0.0088 - val_loss: 1.2490 - val_acc: 0.0120


In [ ]:
predicted_test_scores = predict_batched_models(models, test_features, test_labels, n_batches=n_batches)
print(predicted_test_scores.shape)
print(predicted_test_scores[0:10])

print_metrics(test_labels, predicted_test_scores, is_train=False)

# print(test_labels.argmax(axis=-1).shape, predicted_test_labels.shape)

200000/200000 [==============================] - 5s 26us/step
(200000, 2)
[[0.6906115  0.30938846]
 [0.38188118 0.6181188 ]
 [0.47492638 0.5250736 ]
 [0.53795445 0.46204552]
 [0.6251925  0.37480754]
 [0.42313164 0.5768683 ]
 [0.5102399  0.48976016]
 [0.6140328  0.38596722]
 [0.56436354 0.43563652]
 [0.5009884  0.4990116 ]]
--------test---------
Confusion Matrix:
True Negative = 141514
False Negative = 241
True Positive = 136
False Positive = 58109
f1 score = 0.005
ROC AUC Score = 0.535
(200000, 2)
200000/200000 [==============================] - 5s 26us/step
(200000, 2)
[[0.7946299  0.20537014]
 [0.77235883 0.2276411 ]
 [0.7629972  0.23700283]
 [0.5577578  0.4422422 ]
 [0.66330355 0.33669636]
 [0.60546374 0.39453632]
 [0.5332667  0.4667333 ]
 [0.6668111  0.33318886]
 [0.56311923 0.4368807 ]
 [0.5402631  0.45973682]]
--------test---------
Confusion Matrix:
True Negative = 176718
False Negative = 355
True Positive = 22
False Positive = 22905
f1 score = 0.002
ROC AUC Score = 0.472
200000/